In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
#from langchain.schema import Document

from langchain_core.documents import Document

#from langchain_community.vectorstores import chroma
from langchain_chroma import Chroma
#utility imports
import numpy as np
from typing import List




d:\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sample_docs=[
"""
1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.
Connectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).
Key Contacts: 7022588703 / 6366576503.
The Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, 
designed to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, 
offering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.
 For 2026, the campus reported exceptional placement statistics, with highest packages reaching up to 80 LPA for B.Tech students and 
 average packages around 15 LPA for top branches. The Corporate and Industry Relations (CIR) cell is particularly active here, 
 facilitating recruitment with global giants like Amazon, SAP Labs, and Walmart Global Tech. While smaller in size compared to Coimbatore, 
 the campus provides modern laboratories, design tools, and clean, hygiene-focused hostels that are highly rated by students


""",

"""
2. Coimbatore Campus (Main Campus) 
Location: Nestled in Ettimadai, this is the largest campus, housing over 12,000 students and 1,500 faculty members.
Academic Range: Offers 120+ programs across UG, PG, and Doctoral levels, including Engineering, Business, and Social Work.
Research: Home to an Internal Quality Assurance Cell (IQAC) and numerous specialized research centers.
Key Contacts: 0422-2685509 / 0422-2685510.

The Coimbatore campus, established in 1994, serves as the administrative headquarters of the university and is a sprawling 400-acre "oasis" nestled 
in the Western Ghats at Ettimadai. As of 2026, it supports over 12,000 students and nearly 1,500 faculty members across 120+ undergraduate,
 postgraduate, and doctoral programs. The campus is renowned for its eco-friendly initiatives, including massive forestation that 
 transformed once-barren land into a lush environment home to exotic birds like peacocks. Infrastructure highlights include a built-up 
 area of 28 lakh square feet, eight hostels accommodating 5,000 students, and an Olympic-standard swimming pool (50m x 25m). 
 Academic life is centered around the Schools of Engineering and Business, alongside Departments of Social Work and Communication. 
 For the 2026 placement season, the campus maintains a strong record with top recruiters like Google, Microsoft, and Intel, reporting 
 a highest package of 60 LPA and an average of 9 LPA.
""",

"""
3. Amritapuri Campus 
Location: Located in Vallikavu, Kerala, spanning 80 acres along the coastline.
Structure: Features 6 prestigious schools and over 20 research centers.
Academic Focus: Known for blending science with spirituality, offering programs in Computing, Engineering, Biotechnology, and Ayurveda.
Key Dates (2026): Even semester classes for 2026 are scheduled to commence on January 19, 2026.
Key Contacts: 7034024264 / 0476-2805280.

Situated in the serene coastal village of Vallikavu, Kerala, the Amritapuri campus spans 80 acres and is a unique blend of
 modern science and spiritual values. It features over one million square feet of built-up area, housing six prestigious schools 
 and more than 20 specialized research centers. The campus is highly residential, with approximately 95 percent of students living in hostels, 
 many of which are located within the ashram premises. Academic programs cover diverse fields including Biotechnology, Ayurveda, Engineering,
   and Social Sciences. Amritapuri is a research powerhouse, hosting the Humanitarian Technology (HuT) Labs, which recently launched the IOT-based 
   "PRABHA" robot and is hosting the 2026 IEEE International Conference on Robotics and Mechatronics. Facilities for students include high-speed 
   computer labs with over 600 systems, extensive sports complexes for cricket and football, and strictly vegetarian dining that promotes a holistic
     lifestyle.

"""



]

In [7]:
import os

data_dir = "data1"
os.makedirs(data_dir, exist_ok=True)

for i, doc in enumerate(sample_docs):
    with open(f"{data_dir}/doc_{i}.txt", "w", encoding="utf-8") as f:
        f.write(doc)

print("Documents saved in:", data_dir)


Documents saved in: data1


In [4]:
#STEP1:  -----> DOCUMENT LOADING
from langchain_community.document_loaders import DirectoryLoader,TextLoader

# Load documents from directory
loader = DirectoryLoader(
    "data1", 
    glob="*.txt", 
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)
documents = loader.load()

print(f"Loaded {len(documents)} documents")
print(f"\nFirst document preview:")
print(documents[0].page_content[:200] + "...")


Loaded 3 documents

First document preview:

1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tec...


In [5]:
documents

[Document(metadata={'source': 'data1\\doc_0.txt'}, page_content="\n1. Bengaluru Campus (School of Engineering & Computing) \nLocation: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.\nAcademic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.\nConnectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).\nKey Contacts: 7022588703 / 6366576503.\nThe Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, \ndesigned to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, \noffering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.\n For 2026, the campus reported exceptional placement statistics, with highest packages reaching up to 80 LPA for B

In [6]:
#step2----->DOCUMENT SPLITTING
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Maximum size of each chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    length_function=len,
    separators=[" "]  # Hierarchy of separators
)
chunks=text_splitter.split_documents(documents)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")
print(f"\nChunk example:")
print(f"Content: {chunks[0].page_content[:150]}...")
print(f"Metadata: {chunks[0].metadata}")

Created 11 chunks from 3 documents

Chunk example:
Content: 1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academi...
Metadata: {'source': 'data1\\doc_0.txt'}


In [7]:
chunks

[Document(metadata={'source': 'data1\\doc_0.txt'}, page_content='1. Bengaluru Campus (School of Engineering & Computing) \nLocation: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.\nAcademic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.\nConnectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).\nKey Contacts: 7022588703 / 6366576503.\nThe Bengaluru campus is a 50-acre'),
 Document(metadata={'source': 'data1\\doc_0.txt'}, page_content="/ 6366576503.\nThe Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, \ndesigned to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, \noffering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.\

In [9]:

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

persist_directory = "./chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory,
    collection_name="amrita_clg"
)

vectorstore.persist()

print("Vectors stored:", vectorstore._collection.count())


Vectors stored: 11


C:\Users\DELL\AppData\Local\Temp\ipykernel_15876\1931119460.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [10]:
#just to load--without duplicating the vectors

persist_directory = "./chroma_db"

# Load existing vectorstore
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=OpenAIEmbeddings(),
    collection_name="amrita_clg"
)

print(f"Vector store has {vectorstore._collection.count()} vectors")


Vector store has 11 vectors


C:\Users\DELL\AppData\Local\Temp\ipykernel_15876\3512872745.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [11]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(
    model_name="gpt-5-nano"
)


In [12]:
## Convert vector store to retriever
retriever=vectorstore.as_retriever(
    search_kwargs={"k":3} ## Retrieve top 3 relevant chunks
)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001EE2CE296D0>, search_kwargs={'k': 3})

In [13]:
##CREATING RAG CHAIN USING LCEL
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

In [14]:
# Create a custom prompt
from langchain_core.prompts import ChatPromptTemplate
custom_prompt = ChatPromptTemplate.from_template("""Use the following context to answer the question 
about the college and place. Donot hallucinate and do not guess anything.
be crystal clear to the asked question.
If you don't know the answer based on the context, say you don't know.
give the specific details and support.

Context:
{context}

Question: {query}

Answer:""")
custom_prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="Use the following context to answer the question \nabout the college and place. Donot hallucinate and do not guess anything.\nbe crystal clear to the asked question.\nIf you don't know the answer based on the context, say you don't know.\ngive the specific details and support.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"), additional_kwargs={})])

In [15]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001EE2CE296D0>, search_kwargs={'k': 3})

In [16]:
#output doc for the promts...(combining all the chunks from the retriever for the prompt.)
def combi(docs):
    return "\n".join(doc.page_content for doc in docs)


In [17]:
rag_chain_lcel=(
    {
        "context":retriever|combi,
        "query":RunnablePassthrough()
    }
    |custom_prompt
    |llm
    |StrOutputParser()
)
rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001EE2CE296D0>, search_kwargs={'k': 3})
           | RunnableLambda(combi),
  query: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="Use the following context to answer the question \nabout the college and place. Donot hallucinate and do not guess anything.\nbe crystal clear to the asked question.\nIf you don't know the answer based on the context, say you don't know.\ngive the specific details and support.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': Fal

In [18]:
response=rag_chain_lcel.invoke("what is the location on amritapuri campus in 1000 words")
response

'Based on the provided context, the location of the Amritapuri campus is Vallikavu, Kerala, India, situated along the coastline. Here is a detailed, location-focused description drawn strictly from the given information.\n\n- Geographic setting\n  - The Amritapuri campus is located in Vallikavu, a serene coastal village in the Indian state of Kerala.\n  - It sits along the coastline, leveraging a coastal riverine and marine environment that characterizes the area.\n  - The campus’ geographic context is described as a coastal village setting, contributing to a calm, sea-facing ambiance.\n\n- Campus footprint and land area\n  - The campus spans 80 acres, providing a large, expansive site for academic, residential, and research facilities.\n  - The built environment on campus covers more than one million square feet, indicating a substantial and dense physical footprint occupied by academic buildings, housing, laboratories, and other facilities.\n\n- Institutional composition on site\n  -

In [19]:
#to test query_rag_lcel()

In [20]:
from IPython.display import display, Markdown

display(Markdown(response))


Based on the provided context, the location of the Amritapuri campus is Vallikavu, Kerala, India, situated along the coastline. Here is a detailed, location-focused description drawn strictly from the given information.

- Geographic setting
  - The Amritapuri campus is located in Vallikavu, a serene coastal village in the Indian state of Kerala.
  - It sits along the coastline, leveraging a coastal riverine and marine environment that characterizes the area.
  - The campus’ geographic context is described as a coastal village setting, contributing to a calm, sea-facing ambiance.

- Campus footprint and land area
  - The campus spans 80 acres, providing a large, expansive site for academic, residential, and research facilities.
  - The built environment on campus covers more than one million square feet, indicating a substantial and dense physical footprint occupied by academic buildings, housing, laboratories, and other facilities.

- Institutional composition on site
  - The Amritapuri campus houses six prestigious schools, indicating a multi-disciplinary, school-based organizational structure within the same campus area.
  - In addition to the schools, there are more than 20 specialized research centers located on the campus, underscoring the campus’ role as a research-intensive institution.

- Residential character and community
  - The campus is described as highly residential, with approximately 95 percent of students living in hostels.
  - Many of these hostels are located within the ashram premises, reinforcing the integration of campus life with the spiritual and community environment that characterizes the Amritapuri site.
  - This residential pattern highlights that the location is not just a place for classrooms and laboratories but a living, campus-wide community.

- Academic and research environment linked to the location
  - The campus is noted for blending science with spirituality, a characteristic that is tied to its location and cultural setting in Vallikavu.
  - Academic programs on campus include Computing, Engineering, Biotechnology, and Ayurveda, reflecting a mix of modern science and traditional knowledge that aligns with the campus’s spiritual and intellectual milieu.
  - The Amritapuri site hosts the Humanitarian Technology (HuT) Labs, a research facet rooted in practical, technology-driven humanitarian work, illustrating the campus’ role as a hub of applied research in a coastal, community-oriented setting.
  - The campus has also hosted notable events and initiatives tied to its location, such as the launch of the IOT-based “PRABHA” robot and the hosting of the 2026 IEEE International Conference on Robotics and Mechatronics, signaling its standing as a regional and international center for technology and engineering activity.

- Facilities and infrastructure tied to the location
  - The campus features high-speed computer laboratories with more than 600 systems, indicating substantial on-site computing resources available to students and researchers.
  - Extensive sports complexes for cricket and football are part of the campus infrastructure, contributing to the residential, campus-life experience in a coastal setting.
  - The campus maintains a vegetarian regime for its facilities, which aligns with the spiritual and cultural dimensions associated with its Vallikavu location.

In summary, the Amritapuri campus is physically located in Vallikavu, Kerala, India, directly along the coastline, occupying 80 acres with over a million square feet of built-up space. Its location in a serene coastal village supports a unique blend of modern science and spiritual values, as reflected in its six schools, 20+ research centers, and a highly residential community where roughly 95 percent of students live in hostels, many inside the ashram premises. The coastal setting likely influences the campus atmosphere and lifestyle, while the on-site facilities—ranging from HuT Labs and robotics initiatives to extensive computing resources and sports infrastructure—shape a robust environment for research, education, and holistic campus life.

If you need more precise geographic coordinates or a street/address-style location, that information isn’t provided in the given context.

In [21]:
def query_rag(question):
    print(f"Question:{question}")
    print("-"*50)

    #1)pass the string directly (when using RunnablePassThrough)
    ans=rag_chain_lcel.invoke(question)
    #print(f"answer:{ans}")
    display(Markdown(ans))

    #2)printing the sources for the confirmation
    docs=retriever.invoke(question)
    for i,doc in enumerate(docs):
      print(f"\n source{i+1} \n")
      print(doc.page_content[:200])
    
    

In [22]:
res=query_rag(
  "Amrita Vishwa Vidyapeetham Bengaluru campus Sarjapur Road Kasavanahalli location in 500 words"
)


Question:Amrita Vishwa Vidyapeetham Bengaluru campus Sarjapur Road Kasavanahalli location in 500 words
--------------------------------------------------


Amrita Vishwa Vidyapeetham’s Bengaluru campus, specifically the School of Engineering & Computing, is situated off Sarjapur Road in Kasavanahalli/Junnasandra, Bengaluru. The campus spans 50 acres and is positioned as a dynamic tech hub designed to cater to the needs of India’s Silicon Valley. It is closely tied to Amrita’s emphasis on high-demand technical education and research.

Location and campus footprint
- Address/placement: Bengaluru Campus (School of Engineering & Computing) is located off Sarjapur Road in Kasavanahalli/Junnasandra. The site is described as a 50-acre campus.
- Setting: The campus is portrayed as a tech-forward hub that aligns with the broader Amrita vision of blending rigorous engineering education with cutting-edge computing and research.

Academic programs and focus
- Programs: The Bengaluru campus offers B.Tech programs in five disciplines and M.Tech programs in six disciplines. The academic structure emphasizes Engineering and Computing.
- Specializations: There is a strong emphasis on Computer Science, Artificial Intelligence, and Cybersecurity, reflecting modern, industry-relevant priorities in technology and engineering education.

Connectivity and access
- Distance from central hubs: The campus is approximately 17 kilometers from Majestic Bus Terminus and the City Railway Station, making it accessible to students traveling from various parts of Bengaluru.
- Public transport routes: It is reachable via BMTC routes 340 L (Majestic) and 341 B (KR Market). These routes connect the campus to major city transit nodes, facilitating commuting for students and staff.

Facilities and environment
- The description characterizes the Bengaluru campus as a “dynamic tech hub,” indicating an emphasis on modern facilities, laboratories, and spaces designed to support engineering and computing education and research.
- The campus’s positioning near Sarjapur Road places it in a growing tech corridor, aligning with regional emphasis on information technology and software development.

Key contacts
- For inquiries related to the Bengaluru campus, the provided contact numbers are 7022588703 and 6366576503. These numbers are listed in the context as the campus’s key contacts for information and assistance.

Placement and outcomes (as of 2026)
- The context notes exceptional 2026 placement statistics, with highest packages reaching up to 80 LPA for B.Tech students. While specific distribution of placements across disciplines isn’t detailed, this upper-bound figure signals strong placement performance for the campus in recent years.

Summary
In short, Amrita Vishwa Vidyapeetham’s Bengaluru campus (School of Engineering & Computing) sits on a 50-acre site off Sarjapur Road in Kasavanahalli/Junnasandra. It focuses on engineering and computing disciplines, offering B.Tech programs in five fields and M.Tech programs in six, with notable emphasis on Computer Science, Artificial Intelligence, and Cybersecurity. The campus is well-connected to Bengaluru’s city center via BMTC routes and sits about 17 km from Majestic and the City Railway Station. For prospective students or partners, the listed phone numbers (7022588703, 6366576503) are the primary contact points, and the 2026 data suggests strong placement outcomes with top packages reaching up to 80 LPA for B.Tech graduates.


 source1 

1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tech

 source2 

/ 6366576503.
The Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, 
designed to cater to the needs of India's Silicon Valley. It specializes in high-demand te

 source3 

3. Amritapuri Campus 
Location: Located in Vallikavu, Kerala, spanning 80 acres along the coastline.
Structure: Features 6 prestigious schools and over 20 research centers.
Academic Focus: Known for b


#adding documents (updating)

In [23]:
new_info="""

Amrita Vishwa Vidyapeetham provides comprehensive hostel facilities, with separate accommodation for male and female students 
across its various campuses, including Bengaluru and Coimbatore. Staying in the hostel is generally mandatory for all students 
whose parents do not reside within a 30 km radius of the campus. The rooms are typically shared, with undergraduates often in
 four-sharing rooms during their initial years and postgraduate/final-year students eligible for single or double-occupancy
   rooms based on availability. Each room is furnished with a cot, table, chair, and cupboard for each student. 
The hostel environment emphasizes a disciplined and holistic lifestyle, incorporating activities like yoga and sports, and 
is supervised by a team of wardens and faculty wardens who monitor student well-being and activities. All hostels are equipped
 with amenities such as Wi-Fi, solar water heaters for hot water, and a dedicated study hall or reading room. A major highlight
   (or point of note) is that only pure vegetarian food is served in the mess halls, 
   with non-vegetarian food strictly prohibited within the entire campus premises.
     While some students find the food quality average due to a repeating menu, night cafeterias are available for additional
       options. Strict rules are in place regarding timings, outings (usually limited passes per month), and the prohibition
         of intoxicants, tobacco, and specific electrical appliances like kettles or iron boxes within rooms.
           The annual hostel fees vary, ranging approximately from INR 77,000 to INR 121,000 including mess charges, 
           depending on the campus and room type. Students can find more detailed information and apply through the 
           official Amrita Vishwa Vidyapeetham website.
"""

In [24]:
new_docs=Document(
    page_content=new_info,
    metadata={"source":"manual updation", "topic":"amrita hostels"}

)
new_docs

Document(metadata={'source': 'manual updation', 'topic': 'amrita hostels'}, page_content='\n\nAmrita Vishwa Vidyapeetham provides comprehensive hostel facilities, with separate accommodation for male and female students \nacross its various campuses, including Bengaluru and Coimbatore. Staying in the hostel is generally mandatory for all students \nwhose parents do not reside within a 30 km radius of the campus. The rooms are typically shared, with undergraduates often in\n four-sharing rooms during their initial years and postgraduate/final-year students eligible for single or double-occupancy\n   rooms based on availability. Each room is furnished with a cot, table, chair, and cupboard for each student. \nThe hostel environment emphasizes a disciplined and holistic lifestyle, incorporating activities like yoga and sports, and \nis supervised by a team of wardens and faculty wardens who monitor student well-being and activities. All hostels are equipped\n with amenities such as Wi-Fi,

In [26]:
new_chunks=text_splitter.split_documents([new_docs])
new_chunks

[Document(metadata={'source': 'manual updation', 'topic': 'amrita hostels'}, page_content='Amrita Vishwa Vidyapeetham provides comprehensive hostel facilities, with separate accommodation for male and female students \nacross its various campuses, including Bengaluru and Coimbatore. Staying in the hostel is generally mandatory for all students \nwhose parents do not reside within a 30 km radius of the campus. The rooms are typically shared, with undergraduates often in\n four-sharing rooms during their initial years and postgraduate/final-year students eligible for single or'),
 Document(metadata={'source': 'manual updation', 'topic': 'amrita hostels'}, page_content='students eligible for single or double-occupancy\n   rooms based on availability. Each room is furnished with a cot, table, chair, and cupboard for each student. \nThe hostel environment emphasizes a disciplined and holistic lifestyle, incorporating activities like yoga and sports, and \nis supervised by a team of wardens 

In [27]:
vectorstore

In [28]:
vectorstore.add_documents(new_chunks)

['1e971a17-6598-49e5-96f8-fbd4960d9bed',
 '36a43ace-22ff-4cb9-b990-b0ba99df924a',
 'b29762fe-a0ea-4d20-b2c8-2810adb3e675',
 '0b03f2c0-0e15-4699-8a0e-23e4d15e08db']

In [30]:
print(f"vectorstore has {vectorstore._collection.count()} vectors")

vectorstore has 15 vectors


In [32]:
new_query="give me information about amrita hostels"
answer=rag_chain_lcel.invoke(new_query)
display(Markdown(answer))

Here is what the context provides about Amrita hostels:

- General setup
  - Amrita Vishwa Vidyapeetham offers hostel facilities with separate accommodations for male and female students across its campuses (including Bengaluru and Coimbatore).

- Mandatory housing
  - Staying in the hostel is generally mandatory for students whose parents do not reside within a 30 km radius of the campus.

- Room types and occupancies
  - Rooms are typically shared.
  - Undergraduates usually stay in four-sharing rooms during their initial years.
  - Postgraduate and final-year students may be eligible for single occupancy or, in some cases, limited monthly passes.

- Rules inside rooms
  - Prohibition of intoxicants, tobacco, and certain electrical appliances (e.g., kettles or iron boxes) within rooms.

- Fees
  - The annual hostel fees vary roughly from INR 77,000 to INR 121,000, including mess charges, depending on the campus and room type.

- How to get more details
  - Students can find more detailed information and apply through the official Amrita Vishwa Vidyapeetham website.

- Amritapuri (Kerala) campus details (example of a campus)
  - Located in Vallikavu, Kerala; the Amritapuri campus spans 80 acres.
  - It is highly residential, with about 95 percent of students living in hostels.
  - Many hostels are located within the ashram premises.
  - The campus houses six schools and more than 20 research centers.

If you want campus-specific details (e.g., Bengaluru vs. Coimbatore), I can summarize those separately if you provide the campus you’re interested in.